In [1]:
from PolConfML.search import Search_SrTiO3 as Search, search_extended_config_SrTiO3 as search_extended_config
from PolConfML.model import Network, Dataset
from PolConfML.utils import load_descs, save_descs
import torch
import numpy as np
import time
import matplotlib.pyplot as plt

dtype = torch.float32

In [2]:
path = 'data_SrTiO3/split/'

# train
descs_train, indices_train = load_descs(path+'train/',['S0','S1','S2'])
Y_train = np.array(np.load(path+'train/Y.npy'))
defects_train = np.load(path+'train/defect.npy')
n = [descs_train[site].shape[0] for site in descs_train]

data_train = Dataset(descs_train, indices_train, Y_train, defects_train)
data_train.kernel(0.3)

In [3]:
net = torch.load('data_SrTiO3/model.pt')
Nb = np.loadtxt('data_SrTiO3/POSCAR',skiprows=9,usecols=(0,1,2))[:8]
cell = np.diag(np.loadtxt('data_SrTiO3/POSCAR',skiprows=2,max_rows=3))

In [4]:
start = time.time()
res = {}
s = Search('data_SrTiO3/POSCAR', 4)
site_list = ['S0', 'S1', 'S2']
a, b,c,d = search_extended_config(net,
                                  data_train,
                                  s,
                                  np.array([]),
                                  4, 
                                  Nb[:4], 
                                  cell,
                                  site_list,
                                  30000)
sorted_idxs = np.argsort(b)
res[4] = {'energies':b[sorted_idxs], 'idxs':a[sorted_idxs]}
n_search = 100

for j in range(5,9):
    print(j, 'Nb')
    site_list = ['S0', 'S1', 'S2']
    s = Search('data_SrTiO3/POSCAR', j)
    res_idx = []
    res_en = []
    for i,idxs in enumerate(res[j-1]['idxs'][:n_search]):
        print(i/n_search, end='\r')
        search_idxs = idxs-1
        if np.logical_not(np.any(search_idxs <0)):
            a, b,c,d = search_extended_config(net,
                                              data_train,
                                              s, 
                                              search_idxs,
                                              1, 
                                              Nb[:j],
                                              cell,
                                              site_list,
                                              verbose=False)
            sorted_idx = np.argsort(b)
            b = b[sorted_idx]
            a = a[sorted_idx]
            res_idx.append(a)
            res_en.append(b)
    res_idx = np.concatenate(res_idx)
    res_en = np.concatenate(res_en)

    _, idxs = np.unique(res_en,return_index =True)
    res_en = res_en[idxs]
    res_idx = res_idx[idxs]
    sorted_idxs = np.argsort(res_en)
    res[j] = {'energies':res_en[sorted_idxs], 'idxs':res_idx[sorted_idxs]}

end = time.time()
print(end-start)

0.0
0.03683761365938715
0.0736752273187743
0.11051284097816143
0.1473504546375486
0.18418806829693574
0.22102568195632286
0.25786329561571003
0.2947009092750972
0.3315385229344843
0.3683761365938715
0.4052137502532586
0.4420513639126457
0.47888897757203286
0.5157265912314201
0.5525642048908072
0.5894018185501944
0.6262394322095814
0.6630770458689687
0.6999146595283557
0.736752273187743
0.77358988684713
0.8104275005065172
0.8472651141659043
0.8841027278252914
0.9209403414846786
0.9577779551440657
0.9946155688034529
0.9999987720795447
5 Nb
6 Nb
7 Nb
8 Nb
1283.3972699642181


In [5]:
# correct indexing
res_new = {i:res[i]['idxs']+i for i in res}

In [6]:
for i in res_new:
    print(i,'Nb')
    print(res_new[i][:5])

4 Nb
[[13 27 29 45]
 [13 27 29 35]
 [13 27 29 46]
 [13 27 29 37]
 [ 9 13 27 30]]
5 Nb
[[13 27 29 45 11]
 [13 27 29 35 11]
 [13 27 29 40 11]
 [13 27 38 45 11]
 [13 27 33 35 11]]
6 Nb
[[13 27 29 45 10 37]
 [13 27 29 35 10 37]
 [13 27 29 36 45 46]
 [13 27 29 36 45 44]
 [13 27 29 36 45 31]]
7 Nb
[[13 27 29 36 45 44 11]
 [13 27 29 35 10 44 37]
 [13 27 29 36 45 41 11]
 [13 27 29 35 10 41 37]
 [13 27 29 36 45 44 41]]
8 Nb
[[13 27 29 36 45 10 44 54]
 [13 27 29 36 45 10 44 24]
 [13 27 29 36 45 10 44 66]
 [13 27 29 36 45 10 41 54]
 [13 27 29 36 45 10 44 51]]
